In [1]:
import random, os
import numpy as np
import json
from __future__ import annotations
from typing import Dict, Any, List, Tuple, Optional, Callable
from datetime import datetime, timedelta
import requests
# Load .env from either current dir or parent (for OPENAI_API_KEY, etc.)
try:
    from dotenv import load_dotenv
    load_dotenv(dotenv_path=".env"); load_dotenv(dotenv_path="../.env")
except Exception:
    pass

random.seed(7); np.random.seed(7)

#### How to pass a single tool to LLM?

In [2]:
def get_weather(location: str, units: str) -> int:
    """
    Fetches the current weather for a given location and returns the temperature in the specified units.

    Parameters:
    location (str): The location to fetch the weather for.
    units (str): The units for temperature (celsius or fahrenheit).

    Returns:
    int: The current temperature in the specified units.
    """
    if units not in ['celsius', 'fahrenheit']:
        raise ValueError("Units must be either 'celsius' or 'fahrenheit'.")
    else:
        if units == 'celsius':
            return 22
        else:
            return 72

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Retrieves current weather for the given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country e.g. Bogotá, Colombia"
                    },
                    "units": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "Units the temperature will be returned in."
                    }
                },
                "required": ["location", "units"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
client.chat.completions.create??

Signature:
client.chat.completions.create(
    *,
    messages: 'Iterable[ChatCompletionMessageParam]',
    model: 'Union[str, ChatModel]',
    audio: 'Optional[ChatCompletionAudioParam] | NotGiven' = NOT_GIVEN,
    frequency_penalty: 'Optional[float] | NotGiven' = NOT_GIVEN,
    function_call: 'completion_create_params.FunctionCall | NotGiven' = NOT_GIVEN,
    functions: 'Iterable[completion_create_params.Function] | NotGiven' = NOT_GIVEN,
    logit_bias: 'Optional[Dict[str, int]] | NotGiven' = NOT_GIVEN,
    logprobs: 'Optional[bool] | NotGiven' = NOT_GIVEN,
    max_completion_tokens: 'Optional[int] | NotGiven' = NOT_GIVEN,
    max_tokens: 'Optional[int] | NotGiven' = NOT_GIVEN,
    metadata: 'Optional[Metadata] | NotGiven' = NOT_GIVEN,
    modalities: "Optional[List[Literal['text', 'audio']]] | NotGiven" = NOT_GIVEN,
    n: 'Optional[int] | NotGiven' = NOT_GIVEN,
    parallel_tool_calls: 'bool | NotGiven' = NOT_GIVEN,
    prediction: 'Optional[ChatCompletionPredictionContentParam] |

In [19]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

system_prompt = "Fetch the weather temperature for the user."
user_prompt = "What is the weather in Paris in celsius?"
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]

resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)
msg = resp.choices[0].message
print(msg)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_mUhDsbx0AdkEGnAPUDwUinCG', function=Function(arguments='{"location":"Paris, France","units":"celsius"}', name='get_weather'), type='function')])


In [20]:
if msg.tool_calls:
    tc = msg.tool_calls[0]
    args = json.loads(tc.function.arguments or "{}")
    result = get_weather(**args)

    # append tool call & result
    messages.append({
        "role": "assistant",
        "tool_calls": [
            {"id": tc.id, "type": "function",
             "function": {"name": tc.function.name, "arguments": json.dumps(args)}}
        ]
    })
    messages.append({
        "role": "tool",
        "tool_call_id": tc.id,
        "content": json.dumps({"temperature": result, "units": args["units"]})
    })

    print(messages)

    # 5) Ask the model for the final natural-language answer
    final = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    print(final.choices[0].message.content)
else:
    # Model chose to answer without tools
    print(msg.content)

[{'role': 'system', 'content': 'Fetch the weather temperature for the user.'}, {'role': 'user', 'content': 'What is the weather in Paris in celsius?'}, {'role': 'assistant', 'tool_calls': [{'id': 'call_mUhDsbx0AdkEGnAPUDwUinCG', 'type': 'function', 'function': {'name': 'get_weather', 'arguments': '{"location": "Paris, France", "units": "celsius"}'}}]}, {'role': 'tool', 'tool_call_id': 'call_mUhDsbx0AdkEGnAPUDwUinCG', 'content': '{"temperature": 22, "units": "celsius"}'}]
The current temperature in Paris is 22°C.


#### How to pass multiple tools to LLM?

In [45]:
def get_review(place: str, limit: int = 3) -> dict:
    # Dummy sample
    return {"place": place, "reviews": [f"Review {i+1} for {place}" for i in range(limit)]}

TOOLS_REGISTRY: Dict[str, Callable[[Dict[str, Any]], Any]] = {
    "get_weather": lambda args: get_weather(args["location"], args["units"]),
    "get_review": lambda args: get_review(args["place"], args.get("limit", 3)),
}

TOOLS_SCHEMA = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for a location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "units": {"type": "string", "enum": ["celsius","fahrenheit"]}
                },
                "required": ["location","units"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_review",
            "description": "Fetch reviews for a place.",
            "parameters": {
                "type": "object",
                "properties": {
                    "place": {"type": "string"},
                    "limit": {"type": "integer", "minimum": 1, "maximum": 10}
                },
                "required": ["place"]
            }
        }
    }
]

In [48]:
def run_agent(
    messages: List[Dict[str, Any]],
    tools_schema: List[Dict[str, Any]],
    tool_registry: Dict[str, Callable[[Dict[str, Any]], Any]],
    model: str = "gpt-4o-mini",
    max_steps: int = 8,
    truncate_tool_result_chars: int = 15000,
) -> Dict[str, Any]:
    """
    Loop until the model stops calling tools or max_steps reached.
    Returns {"messages": [...], "final_text": str | None}
    """
    for step in range(max_steps):
        resp = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools_schema,
            tool_choice="auto",
            temperature=0.2,
        )
        msg = resp.choices[0].message

        # If model returns final text (no tool calls), we’re done
        tool_calls = getattr(msg, "tool_calls", None)
        print(tool_calls)
        if not tool_calls:
            return {"messages": messages + [{"role": "assistant", "content": msg.content}], "final_text": msg.content}

        # Append the assistant tool_calls message itself
        messages.append({
            "role": "assistant",
            "tool_calls": [
                {
                    "id": tc.id,
                    "type": "function",
                    "function": {"name": tc.function.name, "arguments": tc.function.arguments}
                } for tc in tool_calls
            ]
        })

        # Execute each requested tool and append the tool results
        for tc in tool_calls:
            name = tc.function.name
            raw_args = tc.function.arguments or "{}"
            try:
                args = json.loads(raw_args)
            except Exception as e:
                tool_result = {"error": f"invalid_json_arguments: {e.__class__.__name__}: {e}", "raw": raw_args}
            else:
                try:
                    fn = tool_registry[name]
                    out = fn(args)
                    tool_result = {"ok": True, "result": out}
                except Exception as e:
                    tool_result = {"ok": False, "error": f"{e.__class__.__name__}: {e}"}

            messages.append({
                "role": "tool",
                "tool_call_id": tc.id,
                "content": json.dumps(tool_result)[:truncate_tool_result_chars],
            })
        
        print(messages)

    # If we exhaust steps, return partial
    return {"messages": messages, "final_text": None}


In [47]:
messages = [
    {"role": "system", "content": "You are a helpful travel assistant. Use tools when needed."},
    {"role": "user", "content": "I'm going to Paris. What's the weather in celsius and give me 2 reviews of the Louvre?"}
]

result = run_agent(messages, TOOLS_SCHEMA, TOOLS_REGISTRY)
print("FINAL:\n", result["final_text"])

[ChatCompletionMessageFunctionToolCall(id='call_MPXbOtqemK5BiksVZSYsCZH5', function=Function(arguments='{"location": "Paris", "units": "celsius"}', name='get_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_qfoEMFXTCfzPWPykwwa21BCX', function=Function(arguments='{"place": "Louvre", "limit": 2}', name='get_review'), type='function')]
[{'role': 'system', 'content': 'You are a helpful travel assistant. Use tools when needed.'}, {'role': 'user', 'content': "I'm going to Paris. What's the weather in celsius and give me 2 reviews of the Louvre?"}, {'role': 'assistant', 'tool_calls': [{'id': 'call_MPXbOtqemK5BiksVZSYsCZH5', 'type': 'function', 'function': {'name': 'get_weather', 'arguments': '{"location": "Paris", "units": "celsius"}'}}, {'id': 'call_qfoEMFXTCfzPWPykwwa21BCX', 'type': 'function', 'function': {'name': 'get_review', 'arguments': '{"place": "Louvre", "limit": 2}'}}]}, {'role': 'tool', 'tool_call_id': 'call_MPXbOtqemK5BiksVZSYsCZH5', 'content': '{"ok": 

#### Project

In [23]:
def _date(s: str) -> datetime:
    """Parse YYYY-MM-DD into a datetime.date object (as datetime)."""
    return datetime.strptime(s, "%Y-%m-%d")

def _geocode_open_meteo(city: str) -> Optional[Dict[str, float]]:
    """Geocode a city name to lat/lon using Open-Meteo's free geocoder."""
    r = requests.get(
        "https://geocoding-api.open-meteo.com/v1/search",
        params={"name": city, "count": 1},
        timeout=20
    )
    r.raise_for_status()
    res = r.json().get("results") or []
    if not res:
        return None
    x = res[0]
    return {"name": x["name"], "lat": x["latitude"], "lon": x["longitude"]}

def get_daily_forecast(city: str, start: str, end: str) -> Dict[str, Any]:
    """
    7-day window forecast for [start, end] using Open-Meteo forecast endpoint.
    Returns daily tmax/tmin and precipitation probability.
    """
    loc = _geocode_open_meteo(city)
    if not loc:
        return {"mode": "forecast", "city": city, "error": "geocoding_failed"}
    r = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": loc["lat"],
            "longitude": loc["lon"],
            "daily": "temperature_2m_max,temperature_2m_min,precipitation_probability_mean,weathercode",
            "start_date": start,
            "end_date": end,
            "timezone": "auto",
        },
        timeout=25
    )
    r.raise_for_status()
    d = r.json().get("daily", {})
    out = []
    for i, day in enumerate(d.get("time", [])):
        out.append({
            "date": day,
            "tmax": d.get("temperature_2m_max", [None])[i],
            "tmin": d.get("temperature_2m_min", [None])[i],
            "pop": d.get("precipitation_probability_mean", [None])[i],  # %
            "wcode": d.get("weathercode", [None])[i],
        })
    return {"mode": "forecast", "city": city, "lat": loc["lat"], "lon": loc["lon"], "days": out}

In [24]:
get_daily_forecast('New York City', '2025-09-13', '2025-09-19')

{'mode': 'forecast',
 'city': 'New York City',
 'lat': 40.71427,
 'lon': -74.00597,
 'days': [{'date': '2025-09-13',
   'tmax': 26.0,
   'tmin': 14.6,
   'pop': 1,
   'wcode': 3},
  {'date': '2025-09-14', 'tmax': 27.1, 'tmin': 16.9, 'pop': 4, 'wcode': 3},
  {'date': '2025-09-15', 'tmax': 26.8, 'tmin': 19.6, 'pop': 2, 'wcode': 3},
  {'date': '2025-09-16', 'tmax': 23.4, 'tmin': 19.3, 'pop': 2, 'wcode': 3},
  {'date': '2025-09-17', 'tmax': 22.8, 'tmin': 19.3, 'pop': 6, 'wcode': 2},
  {'date': '2025-09-18', 'tmax': 23.3, 'tmin': 19.5, 'pop': 8, 'wcode': 2},
  {'date': '2025-09-19', 'tmax': 24.5, 'tmin': 18.8, 'pop': 7, 'wcode': 1}]}

In [40]:
def _wmo_to_condition(
    wcode: int | None,
    precip_mm: float | None,
    snow_mm: float | None,
    cloudcover_mean: float | None
) -> str:
    """
    Map WMO weather code + precip/snow/cloudcover to a coarse label:
    'sunny' | 'cloudy' | 'rainy' | 'snowy'.
    """
    precip = (precip_mm or 0.0)
    snow = (snow_mm or 0.0)

    # Hard overrides by actual measured precip/snow
    if snow >= 1.0:
        return "snowy"
    if precip >= 1.0:
        return "rainy"

    # WMO (fallback if precip is light/zero)
    # 0: clear, 1-3: partly to overcast
    # 45/48: fog, 51-67 drizzle/rain variants, 71-77 snow variants,
    # 80-82 rain showers, 85-86 snow showers, 95-99 thunderstorms
    if wcode is not None:
        if wcode == 0:
            return "sunny"
        if 1 <= wcode <= 3 or wcode in (45, 48):
            # Could refine with cloudcover if available
            if cloudcover_mean is not None and cloudcover_mean <= 25:
                return "sunny"
            return "cloudy"
        if (51 <= wcode <= 67) or (80 <= wcode <= 82) or (95 <= wcode <= 99):
            return "rainy"
        if (71 <= wcode <= 77) or (85 <= wcode <= 86):
            return "snowy"

    # Last resort: use cloudcover if we have it
    if cloudcover_mean is not None:
        return "sunny" if cloudcover_mean <= 25 else "cloudy"

    return "cloudy"


def _archive_range(lat: float, lon: float, start: str, end: str) -> dict:
    """
    Fetch daily weather archive for [start, end] at (lat, lon).

    Returns a dict:
      {
        "latitude": ...,
        "longitude": ...,
        "daily": [
          {
            "date": "YYYY-MM-DD",
            "tmax": float | None,
            "tmin": float | None,
            "precip_mm": float | None,
            "snow_mm": float | None,
            "cloudcover_mean": float | None,
            "weathercode": int | None,
            "condition": "sunny"|"cloudy"|"rainy"|"snowy"
          }, ...
        ]
      }
    """
    # Ask for multiple daily variables. The archive API supports these.
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start,
        "end_date": end,
        "daily": ",".join([
            "temperature_2m_max",
            "temperature_2m_min",
            "precipitation_sum",
            "snowfall_sum",
            "weathercode",
            "cloudcover_mean",
        ]),
        "timezone": "auto",
    }
    r = requests.get("https://archive-api.open-meteo.com/v1/archive", params=params, timeout=30)
    r.raise_for_status()
    j = r.json()

    d = j.get("daily", {})
    times = d.get("time", []) or []
    tmaxs = d.get("temperature_2m_max", []) or []
    tmins = d.get("temperature_2m_min", []) or []
    psums = d.get("precipitation_sum", []) or []
    ssums = d.get("snowfall_sum", []) or []
    wcodes = d.get("weathercode", []) or []
    clouds = d.get("cloudcover_mean", []) or []

    out = []
    n = len(times)
    for i in range(n):
        date = times[i]
        tmax = tmaxs[i] if i < len(tmaxs) else None
        tmin = tmins[i] if i < len(tmins) else None
        pmm  = psums[i] if i < len(psums) else None
        smm  = ssums[i] if i < len(ssums) else None
        w    = wcodes[i] if i < len(wcodes) else None
        cc   = clouds[i] if i < len(clouds) else None

        cond = _wmo_to_condition(
            int(w) if w is not None else None,
            float(pmm) if pmm is not None else None,
            float(smm) if smm is not None else None,
            float(cc)  if cc  is not None else None,
        )

        out.append({
            "date": date,
            "tmax": tmax,
            "tmin": tmin,
            "precip_mm": pmm,
            "snow_mm": smm,
            "cloudcover_mean": cc,
            "weathercode": w,
            "condition": cond
        })

    return {
        "latitude": j.get("latitude"),
        "longitude": j.get("longitude"),
        "daily": out
    }

def get_historical_weather_stats(city: str, start: str, end: str, years_back: int = 5) -> dict:
    """
    For trips > 7 days away: sample the same date range over the last N years.
    Returns wet/fair ratios + condition counts + average tmin/tmax across samples.
    """
    loc = _geocode_open_meteo(city)
    if not loc:
        return {"mode": "historical", "city": city, "error": "geocoding_failed"}

    s_dt = datetime.strptime(start, "%Y-%m-%d")
    e_dt = datetime.strptime(end, "%Y-%m-%d")
    span_days = (e_dt - s_dt).days + 1
    this_year = datetime.now().year

    # Aggregates
    wet_days = fair_days = 0
    cond_counts = {"sunny": 0, "cloudy": 0, "rainy": 0, "snowy": 0}
    tmax_acc = 0.0
    tmin_acc = 0.0
    temp_samples = 0
    total_samples = 0
    per_year = []

    for y in range(1, years_back + 1):
        year = this_year - y
        s_y = s_dt.replace(year=year)
        e_y = s_y + timedelta(days=span_days - 1)
        block = _archive_range(loc["lat"], loc["lon"], s_y.strftime("%Y-%m-%d"), e_y.strftime("%Y-%m-%d"))
        days = block.get("daily", [])

        year_wet = year_fair = 0
        year_cond = {"sunny": 0, "cloudy": 0, "rainy": 0, "snowy": 0}

        for d in days:
            precip = float(d.get("precip_mm") or 0.0) + float(d.get("snow_mm") or 0.0)
            if precip >= 1.0:
                wet_days += 1; year_wet += 1
            else:
                fair_days += 1; year_fair += 1
            total_samples += 1

            cond = d.get("condition", "cloudy")
            if cond in year_cond:
                year_cond[cond] += 1
                cond_counts[cond] += 1

            # temps
            if d.get("tmax") is not None:
                tmax_acc += float(d["tmax"])
                temp_samples += 1
            if d.get("tmin") is not None:
                tmin_acc += float(d["tmin"])

        per_year.append({
            "year": year, "wet": year_wet, "fair": year_fair, "total": year_wet + year_fair,
            "conditions": year_cond
        })

    wet_ratio = (wet_days / total_samples) if total_samples else 0.0
    fair_ratio = (fair_days / total_samples) if total_samples else 0.0
    avg_tmax = (tmax_acc / temp_samples) if temp_samples else None
    avg_tmin = (tmin_acc / temp_samples) if temp_samples else None

    return {
        "mode": "historical",
        "city": city,
        "lat": loc["lat"], "lon": loc["lon"],
        "years_back": years_back,
        "samples": total_samples,
        "summary": {
            "wet_days": wet_days,
            "fair_days": fair_days,
            "wet_ratio": wet_ratio,
            "fair_ratio": fair_ratio,
            "avg_tmax": avg_tmax,
            "avg_tmin": avg_tmin,
            "conditions": cond_counts
        },
        "per_year": per_year
    }

In [26]:
get_historical_weather_stats('New York City', '2025-09-13', '2025-09-19')

{'mode': 'historical',
 'city': 'New York City',
 'lat': 40.71427,
 'lon': -74.00597,
 'years_back': 5,
 'samples': 35,
 'summary': {'wet_days': 8,
  'fair_days': 27,
  'wet_ratio': 0.22857142857142856,
  'fair_ratio': 0.7714285714285715,
  'avg_tmax': 24.72,
  'avg_tmin': 15.977142857142859,
  'conditions': {'sunny': 11, 'cloudy': 12, 'rainy': 12, 'snowy': 0}},
 'per_year': [{'year': 2024,
   'wet': 0,
   'fair': 7,
   'total': 7,
   'conditions': {'sunny': 3, 'cloudy': 4, 'rainy': 0, 'snowy': 0}},
  {'year': 2023,
   'wet': 2,
   'fair': 5,
   'total': 7,
   'conditions': {'sunny': 2, 'cloudy': 2, 'rainy': 3, 'snowy': 0}},
  {'year': 2022,
   'wet': 2,
   'fair': 5,
   'total': 7,
   'conditions': {'sunny': 2, 'cloudy': 3, 'rainy': 2, 'snowy': 0}},
  {'year': 2021,
   'wet': 3,
   'fair': 4,
   'total': 7,
   'conditions': {'sunny': 1, 'cloudy': 0, 'rainy': 6, 'snowy': 0}},
  {'year': 2020,
   'wet': 1,
   'fair': 6,
   'total': 7,
   'conditions': {'sunny': 3, 'cloudy': 3, 'rainy': 